In [1]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

from statsmodels.tsa.stattools import grangercausalitytests

# Global variables and paths

In [2]:
company = 'first_solar'
ticker = 'FSLR'

In [3]:
path_stocks = '../../download_stock_data/' + ticker + '.csv'
path_twitter_sentiment = '../../../../data/team/NLTK Time Series/sentiment_nltk_fixed_final.csv'

# Import Data

## Import Stock Data

In [4]:
stock = pd.read_csv(path_stocks)
stock['Date'] = pd.to_datetime(stock['timestamp'])
stock.set_index('Date', inplace=True);
stock.drop(['timestamp'],axis=1, inplace=True)
stock_close_daily = pd.DataFrame(stock['close'])

## Import Twitter Data

In [5]:
twitter_sentiment = pd.read_csv(path_twitter_sentiment)
twitter_sentiment.drop('Unnamed: 0', axis=1, inplace=True)
twitter_sentiment['Date'] = pd.to_datetime(twitter_sentiment['CreatedAt'].str[:10], errors='coerce')
twitter_sentiment.set_index('Date',inplace=True);
twitter_sentiment.drop('CreatedAt', axis=1, inplace=True)

# Select Company

In [6]:
# select twitter_sentiment data based on company
twitter_sentiment_company = twitter_sentiment[twitter_sentiment['company'] == company]

# Create daily tweet count 

In [7]:
tweets_daily = twitter_sentiment_company.groupby(['Date']).count()
tweets_daily = pd.DataFrame(tweets_daily['compound'])
tweets_daily.rename(columns={'compound':'Daily Tweets'},inplace=True)

# Create Sentiment data

In [8]:
sentiment = twitter_sentiment_company[['neg','pos','compound']]

In [9]:
sentiment_daily = pd.DataFrame(sentiment['neg']).groupby(['Date']).mean()

In [10]:
sentiment_daily.sort_index(ascending=False);

In [11]:
sentiment_daily['pos'] = pd.DataFrame(sentiment['pos']).groupby(['Date']).mean()

In [12]:
sentiment_daily['compound'] = pd.DataFrame(sentiment['compound']).groupby(['Date']).mean()

In [13]:
sentiment_daily.sort_index(ascending=False)

,neg,pos,compound
Date,,,
2021-04-01,0.000000,0.000000,0.000000
2021-03-31,0.017048,0.117333,0.202052
2021-03-30,0.021595,0.124456,0.185378
2021-03-29,0.041579,0.135316,0.187032
2021-03-28,0.005857,0.147524,0.395819
...,...,...,...
2011-01-05,0.000000,0.109556,0.220822
2011-01-04,0.176000,0.132667,0.087533
2011-01-03,0.000000,0.203625,0.278762


In [14]:
# sentiment_daily = sentiment.groupby(['Date']).count()

# Merge Data

In [15]:
data = pd.merge(tweets_daily, sentiment_daily, how='left', left_on=['Date'], right_on=['Date'] )
data = pd.merge(data, stock_close_daily, how='left', left_on=['Date'], right_on=['Date'] )
data.dropna(inplace=True)
data.rename(columns={'Daily Tweets':'daily_tweets','close':'closing_price', 'compound':'compound_sentiment', 'neg':'neg_sentiment','pos':'pos_sentiment'}, inplace=True)

# Export Data 

In [16]:
# name of output csv
output = "./data_" + company + "_"  + ticker + ".csv"

In [17]:
data.to_csv(output)

In [18]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,8,0.000000,0.203625,0.278762,132.78
2011-01-04,3,0.176000,0.132667,0.087533,131.12
2011-01-05,9,0.000000,0.109556,0.220822,131.60
2011-01-06,16,0.091500,0.093312,0.020131,134.53
2011-01-07,26,0.006231,0.016885,0.014558,133.58
...,...,...,...,...,...
2021-03-26,80,0.038038,0.141800,0.218311,85.69
2021-03-29,38,0.041579,0.135316,0.187032,81.61
2021-03-30,79,0.021595,0.124456,0.185378,86.53


# Normalize Data

In [19]:
def normalize_daily_tweets(df):
    max_count = df['daily_tweets'].max()
    min_count = df['daily_tweets'].min()
    df['daily_tweets'] = (df['daily_tweets'] - min_count) / (max_count - min_count)
    return df


def normalize_neg_sentiment(df):
    max_count = df['neg_sentiment'].max()
    min_count = df['neg_sentiment'].min()
    df['neg_sentiment'] = (df['neg_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_pos_sentiment(df):
    max_count = df['pos_sentiment'].max()
    min_count = df['pos_sentiment'].min()
    df['pos_sentiment'] = (df['pos_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_compound_sentiment(df):
    max_count = df['compound_sentiment'].max()
    min_count = df['compound_sentiment'].min()
    df['compound_sentiment'] = (df['compound_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_closing_price(df):
    max_count = df['closing_price'].max()
    min_count = df['closing_price'].min()
    df['closing_price'] = (df['closing_price'] - min_count) / (max_count - min_count)
    return df

In [20]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,8,0.000000,0.203625,0.278762,132.78
2011-01-04,3,0.176000,0.132667,0.087533,131.12
2011-01-05,9,0.000000,0.109556,0.220822,131.60
2011-01-06,16,0.091500,0.093312,0.020131,134.53
2011-01-07,26,0.006231,0.016885,0.014558,133.58
...,...,...,...,...,...
2021-03-26,80,0.038038,0.141800,0.218311,85.69
2021-03-29,38,0.041579,0.135316,0.187032,81.61
2021-03-30,79,0.021595,0.124456,0.185378,86.53


In [21]:
data_normalized = normalize_daily_tweets(data)

In [22]:
data_normalized = normalize_neg_sentiment(data)

In [23]:
data_normalized = normalize_pos_sentiment(data)

In [24]:
data_normalized = normalize_compound_sentiment(data)

In [25]:
data_normalized = normalize_closing_price(data)

In [26]:
data_normalized

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,0.008274,0.000000,0.607965,0.674062,0.760926
2011-01-04,0.002364,0.712551,0.396104,0.458135,0.750487
2011-01-05,0.009456,0.000000,0.327101,0.608638,0.753506
2011-01-06,0.017730,0.370445,0.278604,0.382027,0.771930
2011-01-07,0.029551,0.025226,0.050413,0.375734,0.765956
...,...,...,...,...,...
2021-03-26,0.093381,0.153998,0.423374,0.605803,0.464818
2021-03-29,0.043735,0.168336,0.404014,0.570483,0.439162
2021-03-30,0.092199,0.087429,0.371589,0.568617,0.470100


In [27]:
output_normalized = "./data_" + company + "_"  + ticker + "_normalized.csv"
data.to_csv(output_normalized)
data_normalized

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,0.008274,0.000000,0.607965,0.674062,0.760926
2011-01-04,0.002364,0.712551,0.396104,0.458135,0.750487
2011-01-05,0.009456,0.000000,0.327101,0.608638,0.753506
2011-01-06,0.017730,0.370445,0.278604,0.382027,0.771930
2011-01-07,0.029551,0.025226,0.050413,0.375734,0.765956
...,...,...,...,...,...
2021-03-26,0.093381,0.153998,0.423374,0.605803,0.464818
2021-03-29,0.043735,0.168336,0.404014,0.570483,0.439162
2021-03-30,0.092199,0.087429,0.371589,0.568617,0.470100


In [28]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-03,0.008274,0.000000,0.607965,0.674062,0.760926
2011-01-04,0.002364,0.712551,0.396104,0.458135,0.750487
2011-01-05,0.009456,0.000000,0.327101,0.608638,0.753506
2011-01-06,0.017730,0.370445,0.278604,0.382027,0.771930
2011-01-07,0.029551,0.025226,0.050413,0.375734,0.765956
...,...,...,...,...,...
2021-03-26,0.093381,0.153998,0.423374,0.605803,0.464818
2021-03-29,0.043735,0.168336,0.404014,0.570483,0.439162
2021-03-30,0.092199,0.087429,0.371589,0.568617,0.470100
